In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test7_1.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test4_14.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test15_18.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test15_27.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test8_23.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test28_14.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test25_5.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test29_18.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test8_24.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test4_21.jpg
/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/test_22.jpg
/kaggle/input/accident-detection-from-cctv-footage/d

In [2]:
"""
%pip install -U ray
%pip install -U ipywidgets
%pip install torchmetrics
"""

'\n%pip install -U ray\n%pip install -U ipywidgets\n%pip install torchmetrics\n'

In [3]:
# for data manipulation:
import pandas as pd
import numpy as np
import os

# for visualizations:
import matplotlib.pyplot as plt
import seaborn as sns

# for importing and transforming training data:
import torchvision
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder, DatasetFolder, MNIST
from torch.utils.data import Dataset, DataLoader, random_split
from ignite.metrics import Fbeta
from ignite.metrics.confusion_matrix import ConfusionMatrix

# for building the Neural network:
import torch
from torch import nn
from torch import optim
import math

# for hyperparameter tuning using Optuna
import optuna

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Hyper-parameters to tune:
* optimizer
* learning rate
* initializations
* activation functions
* number of hidden layers
* number of neurons in each hidden layer


In [4]:
#defining transformations to use on images

transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(), 
        transforms.Resize((256,256)),
        transforms.ToTensor(),
     transforms.Normalize((0.5,),(0.5,))
])

# Reading images, transforming them, loading them into tensors

# Train data

In [5]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/train/Accident"  

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/train/Accident/" + i for i in accident_images]


train_dat = torch.empty(size = (0,256*256), dtype = torch.uint8)
labels_train = torch.empty(size = (0,1), dtype = torch.int64)

for i in accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([1])
    
    # appending to train_dat
    train_dat = torch.cat((train_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_train = torch.cat((labels_train,label.view(1,1)), axis = 0)
                             
print(train_dat.shape, labels_train.shape)

torch.Size([369, 65536]) torch.Size([369, 1])


In [6]:
# defining the path for accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/train/Non Accident"

# getting  list of all images in the accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/train/Non Accident/" + i for i in non_accident_images]
for i in non_accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([0])
    
    # appending to train_dat
    train_dat = torch.cat((train_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_train = torch.cat((labels_train,label.view(1,1)), axis = 0)
                             
print(train_dat.shape, labels_train.shape)

torch.Size([791, 65536]) torch.Size([791, 1])


# Validation data

In [7]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/val/Accident"  

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/val/Accident/" + i for i in accident_images]

val_dat = torch.empty(size = (0,256*256), dtype = torch.uint8)
labels_val = torch.empty(size = (0,1), dtype = torch.int64)

for i in accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([1])
    
    # appending to train_dat
    val_dat = torch.cat((val_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_val = torch.cat((labels_val,label.view(1,1)), axis = 0)
                             
print(val_dat.shape, labels_val.shape)

torch.Size([46, 65536]) torch.Size([46, 1])


In [8]:
# defining the path for non accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident"  

# getting  list of all images in the non accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/val/Non Accident/" + i for i in non_accident_images]

for i in non_accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([0])
    
    # appending to train_dat
    val_dat = torch.cat((val_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_val = torch.cat((labels_val,label.view(1,1)), axis = 0)
                             
print(val_dat.shape, labels_val.shape)

torch.Size([98, 65536]) torch.Size([98, 1])


# Test data

In [9]:
# defining the path for accident images:
path_for_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/test/Accident" 

# getting  list of all images in the accident images available in the dataset
accident_images = os.listdir(path_for_accident_images)

accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/test/Accident/" + i for i in accident_images]

test_dat = torch.empty(size = (0,256*256), dtype = torch.uint8)
labels_test = torch.empty(size = (0,1), dtype = torch.int64)

for i in accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([1])
    
    # appending to train_dat
    test_dat = torch.cat((test_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_test = torch.cat((labels_test,label.view(1,1)), axis = 0)
                             
print(test_dat.shape, labels_test.shape)

torch.Size([47, 65536]) torch.Size([47, 1])


In [10]:
# defining the path for non accident images:
path_for_non_accident_images = "/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident" 

# getting  list of all images in the non accident images available in the dataset
non_accident_images = os.listdir(path_for_non_accident_images)

non_accident_images_paths = ["/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident/" + i for i in non_accident_images]
non_accident_images_paths.remove('/kaggle/input/accident-detection-from-cctv-footage/data/test/Non Accident/Pictures - Shortcut.lnk')

for i in non_accident_images_paths:
    
    # reading the image
    t = torchvision.io.read_image(i, mode = torchvision.io.ImageReadMode.GRAY)
    t = transform(t)
    
    # label for this image
    label = torch.tensor([0])
    
    # appending to train_dat
    test_dat = torch.cat((test_dat,t.view(1,-1)), axis = 0)
    
    # appending label
    labels_test = torch.cat((labels_test,label.view(1,1)), axis = 0)
                             
print(test_dat.shape, labels_test.shape)

torch.Size([100, 65536]) torch.Size([100, 1])


# Defining `CustomDataset` to build data loaders

In [11]:
# defining a class to load the training set in batches of size 64 each

class CustomDataset(Dataset):
    def __init__(self, data, labels, transform_list=None):
        self.data = data 
        self.labels = labels
        self.transform = transform_list
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        image = self.data[idx].reshape(256,256)
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [12]:
# train data loader    
final_dat = CustomDataset(train_dat, labels_train)      
trainloader = DataLoader(final_dat, batch_size = 64, shuffle = True)

# validation data loader
final_dat = CustomDataset(val_dat, labels_val)      
valloader = DataLoader(final_dat, batch_size = 64, shuffle = True)

# test data loader
final_dat = CustomDataset(test_dat, labels_test)      
testloader = DataLoader(final_dat, batch_size = 64, shuffle = True)

# Defining `device` to use

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Defining the network

In [14]:
# defining the network:
class Net(nn.Module):
    def __init__(self, input_size, output_size, activation_fn, hidden_size,num_hidden_layers,initial):
        super(Net, self).__init__()
        
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layers = nn.ModuleList([
            nn.Linear(hidden_size, hidden_size) for _ in range(num_hidden_layers)
        ])
        
        self.output_layer = nn.Linear(hidden_size,output_size)
        
        def get_activation(activation_fn):
            activ_dict = {'elu':nn.ELU(), 'leaky-relu' : nn.LeakyReLU(), 'prelu': nn.PReLU(), 'relu': nn.ReLU(),
                          'rrelu': nn.RReLU(), 'selu':  nn.SELU(), 'gelu': nn.GELU(), 
                          'sigmoid' : nn.Sigmoid(), 'tanh': nn.Tanh()}
            return activ_dict[activation_fn]
        
        self.activation = get_activation(activation_fn)
        
        self.initialize_weights(initial)
        self.input_size = input_size
        self.output_size = output_size
        
    def initialize_weights(self, initial):
        initialization = initial
        for m in self.modules():
            if isinstance(m, nn.Linear):
                if initialization == 'Glorot_unif':
                    nn.init.xavier_uniform_(m.weight)
                if initialization == 'He_unif':
                    nn.init.kaiming_uniform_(m.weight)
                if initialization == 'Glorot_norm':
                    nn.init.xavier_normal_(m.weight)
                if initialization == 'He_norm':
                    nn.init.kaiming_normal_(m.weight)    
        
    
    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.input_layer(x)
        for hidden_layer in self.hidden_layers:
            x = self.activation(hidden_layer(x))
       
        x =  nn.LogSoftmax(dim = 1)(self.output_layer(x))
        
        return x
    
# function to count number of parameters in the neural network:

def get_n_params(model):
    np = 0
    for p in list(model.parameters()):
        np += p.nelement()
    return np    
        

# Function to be used to train in **every** epoch

In [15]:
input_size = 256*256
output_size = 2

def train(epoch, model, optimizer, lr, beta1, beta2, rmsprop_alpha, momentum):
    
    model.train()
    criterion = nn.NLLLoss(reduction = 'sum')
    
    if optimizer == 'SGD':
        lr = lr
        optimizer= optim.SGD(model.parameters(),lr = lr)
        
    elif optimizer == 'SGD with momentum':
        lr = lr
        momentum = momentum
        optimizer= optim.SGD(model.parameters(),lr = lr, momentum = momentum)
        
    elif optimizer == 'SGD with nesterov momentum':
        lr = lr
        momentum = momentum
        optimizer= optim.SGD(model.parameters(),lr = lr, momentum = momentum, nesterov = True)
        
    elif optimizer == 'Adagrad':
        lr = lr
        optimizer= optim.Adagrad(model.parameters(),lr = lr)
        
    elif optimizer == 'RMSprop':
        lr = lr
        alpha = rmsprop_alpha
        optimizer= optim.RMSprop(model.parameters(),lr = lr, alpha = alpha)
        
    elif optimizer == 'Adam':
        beta1 = beta1
        beta2 = beta2
        lr = lr
        optimizer= optim.Adam(model.parameters(),lr = lr, betas = (beta1, beta2))
        
    elif optimizer == 'Adamax':
        beta1 = beta1
        beta2 = beta2
        lr = lr
        optimizer= optim.Adamax(model.parameters(),lr = lr, betas = (beta1, beta2))
    
     
    elif optimizer == 'NAdam':
        beta1 = beta1
        beta2 = beta2
        lr = lr
        optimizer= optim.NAdam(model.parameters(),lr = lr, betas = (beta1, beta2))
    
    running_loss = 0.0
    epoch_steps = 0
    for _, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.view(labels.size(0))

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        
        # clipping gradients
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        epoch_steps += 1

    return running_loss/len(trainloader)
        

# Function to keep track of `validation loss` for each tried combination

In [48]:
def val_loss(net):
    
    criterion = nn.NLLLoss(reduction = 'sum')
    val_loss = 0
    val_steps = 0
    total = 0
    correct = 0

    for _, data in enumerate(valloader):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.view(labels.size(0))
            outputs = net.forward(inputs)
            predicted = torch.max(outputs.data, dim = 1, keepdim = True)[1]
            total = labels.size(0)
            correct += predicted.eq(labels.data.view_as(predicted)).cpu().sum().item()
            loss = criterion(outputs, labels)

            val_loss += loss.cpu().numpy()
            val_steps += 1

    print("Validation loss {}".format(val_loss/len(valloader)))  
    return 
                            

# Function to calculate metric of interest i.e., `fbeta score` for every combination

In [49]:
def val_metric(net):
    fbeta = Fbeta(beta = 1.4)
    with torch.no_grad():
        for data in valloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            fbeta.update((predicted, labels.data.view_as(predicted)))
        result = fbeta.compute()
        
    print('Final fbeta: {}'.format(result))    
    return result


# Function to choose combinations and output `fbeta scores`

In [37]:
def objective(trial):
    torch.manual_seed(40)
    params = {"lr": trial.suggest_float('lr',low = 1e-4, high = 1e-1),
        "optimizer": trial.suggest_categorical('optimizer',['SGD','SGD with momentum','SGD with nesterov momentum',
                                                'Adagrad','RMSprop','Adam','Adamax','NAdam']),
        "initialization": trial.suggest_categorical('initialization',['Glorot_unif','Glorot_norm','He_unif','He_norm']),
        "activation": trial.suggest_categorical("activation",['elu','leaky-relu', 'prelu','relu','rrelu','selu', 'gelu','sigmoid', 'tanh']),
        "momentum": trial.suggest_float('momentum', 0.01, 1),
        "beta1" : trial.suggest_float('beta1',0.01, 1),
        "beta2" : trial.suggest_float('beta2',0.01, 1),
        "rmsprop_alpha": trial.suggest_float('rmsprop_alpha',0.01, 1),
        'hidden_size': trial.suggest_int('hidden_size',50,300),
        'num of hidden layers' : trial.suggest_int('num of hidden layers',1,5)}
    
    
    print("Training on ", device)
    model_fnn = Net(input_size, output_size, activation_fn = params['activation'], hidden_size = params['hidden_size'],
                 num_hidden_layers = params['num of hidden layers'], initial = params['initialization'])
    
    model_fnn.to(device)
    print('Number of parameters: {}'.format(get_n_params(model_fnn)))
    
    for epoch in range(0, 10):
        train_loss = train(epoch, model_fnn, optimizer = params['optimizer'], lr = params['lr'], momentum = params['momentum'], 
            beta1 = params['beta1'], beta2 = params['beta2'], rmsprop_alpha = params['rmsprop_alpha'])
    
    print('Done, loss: {}'.format(train_loss))  # last epoch's loss printed
    val_loss(model_fnn)    
    out = val_metric(model_fnn)
        
    return out
    

In [50]:
torch.manual_seed(40)

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 500)
print('\n-------------------------------------------------------------------------------------------------------------------------\n')
print('best_trial:')
trial_ = study.best_trial
print('Best fbeta score on validation set: {}'.format(round(trial_.values[0],2)))
print('\nBest parameters')
print(trial_.params)

[I 2023-12-21 14:11:15,125] A new study created in memory with name: no-name-aad59118-f711-43b9-b2d7-6a90ae673728


Training on  cuda
Number of parameters: 10051031


[I 2023-12-21 14:11:16,790] Trial 0 finished with value: 0.8446981075398015 and parameters: {'lr': 0.06736190411564086, 'optimizer': 'SGD with nesterov momentum', 'initialization': 'Glorot_unif', 'activation': 'gelu', 'momentum': 0.3554387697474803, 'beta1': 0.13994436972631197, 'beta2': 0.3098385805887131, 'rmsprop_alpha': 0.2382198812813787, 'hidden_size': 153, 'num of hidden layers': 1}. Best is trial 0 with value: 0.8446981075398015.


Done, loss: 13.45675484950726
Validation loss 13.35698938369751
Final fbeta: 0.8446981075398015
Training on  cuda
Number of parameters: 5010226


[I 2023-12-21 14:11:18,145] Trial 1 finished with value: 0.9034406215316313 and parameters: {'lr': 0.05641550435960407, 'optimizer': 'SGD with momentum', 'initialization': 'Glorot_unif', 'activation': 'elu', 'momentum': 0.39839307529184453, 'beta1': 0.18864758253181696, 'beta2': 0.2668112698950483, 'rmsprop_alpha': 0.2640319182544163, 'hidden_size': 76, 'num of hidden layers': 5}. Best is trial 1 with value: 0.9034406215316313.


...

Done, loss: 14.318737323467548
Validation loss 18.069706439971924
Final fbeta: 0.8775457592162926
Training on  cuda
Number of parameters: 11464688


[I 2023-12-21 14:25:00,532] Trial 498 finished with value: 0.7977171845275839 and parameters: {'lr': 0.051998890039390414, 'optimizer': 'SGD', 'initialization': 'Glorot_unif', 'activation': 'sigmoid', 'momentum': 0.05782419550033599, 'beta1': 0.23863204738074348, 'beta2': 0.44294228201305713, 'rmsprop_alpha': 0.12602312541506372, 'hidden_size': 174, 'num of hidden layers': 2}. Best is trial 381 with value: 0.9635986978395974.


Done, loss: 27.02173453110915
Validation loss 21.72219753265381
Final fbeta: 0.7977171845275839
Training on  cuda
Number of parameters: 11894762


[I 2023-12-21 14:25:02,141] Trial 499 finished with value: 0.9045915324985089 and parameters: {'lr': 0.056055948167605826, 'optimizer': 'SGD', 'initialization': 'Glorot_unif', 'activation': 'leaky-relu', 'momentum': 0.031227536069715834, 'beta1': 0.15307366160938884, 'beta2': 0.47225210008869206, 'rmsprop_alpha': 0.16417613161174066, 'hidden_size': 180, 'num of hidden layers': 3}. Best is trial 381 with value: 0.9635986978395974.


Done, loss: 15.212023184849667
Validation loss 11.68431806564331
Final fbeta: 0.9045915324985089

-------------------------------------------------------------------------------------------------------------------------

best_trial:
Best fbeta score on validation set: 0.96

Best parameters
{'lr': 0.04380140775602533, 'optimizer': 'SGD', 'initialization': 'Glorot_unif', 'activation': 'leaky-relu', 'momentum': 0.04078724239379571, 'beta1': 0.30813818527966164, 'beta2': 0.5835868146365623, 'rmsprop_alpha': 0.0872345069918592, 'hidden_size': 158, 'num of hidden layers': 2}


In [51]:
best_params = trial_.params
final_model = Net(input_size, output_size, activation_fn = best_params['activation'], hidden_size = best_params['hidden_size'],
                  num_hidden_layers = best_params['num of hidden layers'],initial = best_params['initialization'])
final_model.to(device)
for epoch in range(0, 10):
        train_loss = train(epoch, final_model, optimizer = best_params['optimizer'], lr = best_params['lr'],
                           momentum = best_params['momentum'], beta1 = best_params['beta1'], beta2 = best_params['beta2'],
                           rmsprop_alpha = best_params['rmsprop_alpha'])

# Calculating loss and `fbeta` on test set

In [52]:
def test_loss(net):
    
    criterion = nn.NLLLoss(reduction = 'sum')
    val_loss = 0
    val_steps = 0
    total = 0
    correct = 0

    for _, data in enumerate(testloader):

        with torch.no_grad():
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.view(labels.size(0))
            outputs = net.forward(inputs)
            predicted = torch.max(outputs.data, dim = 1, keepdim = True)[1]
            total = labels.size(0)
            correct += predicted.eq(labels.data.view_as(predicted)).cpu().sum().item()
            loss = criterion(outputs, labels)

            val_loss += loss.cpu().numpy()
            val_steps += 1

    print("Test loss {}".format(val_loss/len(testloader)))  
    return 
                            

In [70]:
def test_metric(net):
    
    fbeta = Fbeta(beta = 1.4)
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            fbeta.update((predicted, labels.data.view_as(predicted)))
        result1 = fbeta.compute()
        
    print('Final fbeta on test set: {}'.format(round(result1,4))) 
    
    cm = ConfusionMatrix(num_classes = 2)
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net.forward(images)
            labels = labels.view((labels.shape[0],))
         #   _, predicted = torch.max(outputs.data, 1)
            cm.update((outputs, labels))
        result2 = cm.compute()
        
    print('Final confusion matrix on test set: {}'.format(result2)) 
    


In [60]:
test_loss(final_model)

Test loss 12.20743465423584


In [71]:
test_metric(final_model)

Final fbeta on test set: 0.9049
Final confusion matrix on test set: tensor([[52,  1],
        [ 6, 41]])
